# Введение в RNN и LSTM

В этом notebook мы познакомимся с рекуррентными нейронными сетями (RNN) и Long Short-Term Memory (LSTM) сетями, которые широко применяются для анализа последовательностей в биологии: ДНК, РНК, белковых последовательностей, временных рядов экспрессии генов и др.

## 1. Установка библиотек

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Что такое RNN?

**Рекуррентная нейронная сеть (RNN)** — это тип нейронной сети, специально разработанный для работы с последовательностями данных. В отличие от обычных нейронных сетей, RNN имеет "память" — она может учитывать предыдущие элементы последовательности при обработке текущего элемента.

### Применение в биологии:
- Предсказание структуры белков
- Анализ последовательностей ДНК/РНК
- Предсказание сайтов связывания транскрипционных факторов
- Анализ временных рядов экспрессии генов
- Классификация последовательностей (например, определение семейств белков)

## 3. Простой пример: Vanilla RNN

Создадим простую RNN для классификации последовательностей ДНК.

In [ ]:
# Простая RNN модель
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # RNN слой
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        
        # Полносвязный слой для выхода
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Инициализация скрытого состояния
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Прямой проход через RNN
        out, _ = self.rnn(x, h0)
        
        # Берем выход последнего временного шага
        out = self.fc(out[:, -1, :])
        return out

## 4. Пример данных: кодирование ДНК последовательностей

Создадим синтетические данные для демонстрации. В реальных задачах вы будете использовать реальные последовательности.

In [ ]:
# One-hot кодирование нуклеотидов
nucleotide_dict = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

def encode_sequence(seq):
    """Преобразует ДНК последовательность в one-hot encoding"""
    encoding = torch.zeros(len(seq), 4)
    for i, nucleotide in enumerate(seq):
        if nucleotide in nucleotide_dict:
            encoding[i, nucleotide_dict[nucleotide]] = 1
    return encoding

# Пример
example_seq = "ATCGATCG"
encoded = encode_sequence(example_seq)
print(f"Последовательность: {example_seq}")
print(f"Размерность encoded: {encoded.shape}")
print(f"\nПервые 3 нуклеотида (one-hot):\n{encoded[:3]}")

## 5. Создание синтетического датасета

Создадим задачу бинарной классификации: последовательности, начинающиеся с "ATG" (старт-кодон), будут иметь метку 1, остальные — 0.

In [ ]:
import random

def generate_dna_sequence(length=50, start_with_atg=False):
    """Генерирует случайную ДНК последовательность"""
    nucleotides = ['A', 'C', 'G', 'T']
    if start_with_atg:
        seq = 'ATG' + ''.join(random.choices(nucleotides, k=length-3))
    else:
        # Убедимся, что не начинается с ATG
        seq = ''.join(random.choices(nucleotides, k=length))
        while seq.startswith('ATG'):
            seq = ''.join(random.choices(nucleotides, k=length))
    return seq

# Создаем датасет
def create_dataset(n_samples=1000, seq_length=50):
    sequences = []
    labels = []
    
    for _ in range(n_samples // 2):
        # Положительные примеры (с ATG)
        seq = generate_dna_sequence(seq_length, start_with_atg=True)
        sequences.append(encode_sequence(seq))
        labels.append(1)
        
        # Отрицательные примеры (без ATG)
        seq = generate_dna_sequence(seq_length, start_with_atg=False)
        sequences.append(encode_sequence(seq))
        labels.append(0)
    
    return torch.stack(sequences), torch.tensor(labels, dtype=torch.long)

# Создаем тренировочные и тестовые данные
X_train, y_train = create_dataset(n_samples=800, seq_length=50)
X_test, y_test = create_dataset(n_samples=200, seq_length=50)

print(f"Размер тренировочных данных: {X_train.shape}")
print(f"Размер меток: {y_train.shape}")
print(f"Распределение классов (train): {torch.bincount(y_train)}")

## 6. Обучение простой RNN

In [ ]:
# Гиперпараметры
input_size = 4  # 4 нуклеотида
hidden_size = 32
output_size = 2  # бинарная классификация
num_epochs = 20
learning_rate = 0.001
batch_size = 32

# Создаем модель
model_rnn = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_rnn.parameters(), lr=learning_rate)

# DataLoader
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Обучение
train_losses = []

for epoch in range(num_epochs):
    model_rnn.train()
    epoch_loss = 0
    
    for sequences, labels in train_loader:
        # Прямой проход
        outputs = model_rnn(sequences)
        loss = criterion(outputs, labels)
        
        # Обратный проход
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# График loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses)
plt.title('Training Loss (RNN)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

## 7. Оценка модели RNN

In [ ]:
# Оценка на тестовых данных
model_rnn.eval()
with torch.no_grad():
    outputs = model_rnn(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy RNN на тестовых данных: {accuracy * 100:.2f}%')

# Матрица ошибок
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

cm = confusion_matrix(y_test.numpy(), predicted.numpy())
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (RNN)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print("\nClassification Report:")
print(classification_report(y_test.numpy(), predicted.numpy(), target_names=['No ATG', 'Has ATG']))

## 8. Проблема исчезающего градиента в RNN

Простые RNN страдают от **проблемы исчезающего/взрывающегося градиента** при работе с длинными последовательностями. Это затрудняет обучение сети запоминать информацию на больших расстояниях.

**Решение: LSTM (Long Short-Term Memory)**

## 9. LSTM архитектура

**LSTM** — это специальный тип RNN, который решает проблему долгосрочных зависимостей с помощью:
- **Ячейки состояния (cell state)** — "память" сети
- **Вентилей (gates)**:
  - *Forget gate* — решает, что забыть из предыдущего состояния
  - *Input gate* — решает, какую новую информацию сохранить
  - *Output gate* — решает, что выдать на выход

Эти механизмы позволяют LSTM эффективно запоминать важную информацию и забывать неважную.

In [ ]:
# LSTM модель
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM слой
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Полносвязный слой
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Инициализация скрытого состояния и cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Прямой проход через LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Берем выход последнего временного шага
        out = self.fc(out[:, -1, :])
        return out

## 10. Обучение LSTM

In [ ]:
# Создаем LSTM модель
model_lstm = LSTMModel(input_size, hidden_size, output_size, num_layers=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=learning_rate)

# Обучение
train_losses_lstm = []

for epoch in range(num_epochs):
    model_lstm.train()
    epoch_loss = 0
    
    for sequences, labels in train_loader:
        outputs = model_lstm(sequences)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses_lstm.append(avg_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Сравнение loss между RNN и LSTM
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='RNN', linewidth=2)
plt.plot(train_losses_lstm, label='LSTM', linewidth=2)
plt.title('Training Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## 11. Оценка LSTM модели

In [ ]:
# Оценка LSTM
model_lstm.eval()
with torch.no_grad():
    outputs = model_lstm(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy LSTM на тестовых данных: {accuracy * 100:.2f}%')

# Матрица ошибок
cm = confusion_matrix(y_test.numpy(), predicted.numpy())
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
plt.title('Confusion Matrix (LSTM)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

print("\nClassification Report:")
print(classification_report(y_test.numpy(), predicted.numpy(), target_names=['No ATG', 'Has ATG']))

## 12. Bidirectional LSTM (BiLSTM)

**Двунаправленная LSTM** обрабатывает последовательность в обоих направлениях (слева направо и справа налево), что позволяет учитывать контекст с обеих сторон. Это особенно полезно для задач, где важна информация из будущих элементов последовательности.

In [ ]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Bidirectional LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, bidirectional=True)
        
        # Выходной слой (hidden_size * 2 из-за bidirectional)
        self.fc = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, x):
        # num_directions = 2 для bidirectional
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Создаем и обучаем BiLSTM
model_bilstm = BiLSTMModel(input_size, hidden_size, output_size, num_layers=2)
optimizer = optim.Adam(model_bilstm.parameters(), lr=learning_rate)

train_losses_bilstm = []

for epoch in range(num_epochs):
    model_bilstm.train()
    epoch_loss = 0
    
    for sequences, labels in train_loader:
        outputs = model_bilstm(sequences)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses_bilstm.append(avg_loss)
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

# Оценка BiLSTM
model_bilstm.eval()
with torch.no_grad():
    outputs = model_bilstm(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'\nAccuracy BiLSTM на тестовых данных: {accuracy * 100:.2f}%')

## 13. Сравнение всех моделей

In [ ]:
# Сравнение loss всех моделей
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='RNN', linewidth=2)
plt.plot(train_losses_lstm, label='LSTM', linewidth=2)
plt.plot(train_losses_bilstm, label='BiLSTM', linewidth=2)
plt.title('Training Loss Comparison: RNN vs LSTM vs BiLSTM')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Сравнение accuracy
models = ['RNN', 'LSTM', 'BiLSTM']
accuracies = []

for model in [model_rnn, model_lstm, model_bilstm]:
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)
        acc = (predicted == y_test).sum().item() / len(y_test)
        accuracies.append(acc)

plt.figure(figsize=(10, 6))
plt.bar(models, accuracies, color=['skyblue', 'lightgreen', 'coral'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
for i, acc in enumerate(accuracies):
    plt.text(i, acc + 0.02, f'{acc*100:.2f}%', ha='center', fontsize=12)
plt.show()

## 14. Практический пример: предсказание для новой последовательности

In [ ]:
def predict_sequence(model, sequence_str):
    """Предсказывает класс для заданной последовательности"""
    model.eval()
    
    # Кодируем последовательность
    encoded = encode_sequence(sequence_str).unsqueeze(0)  # добавляем batch dimension
    
    with torch.no_grad():
        output = model(encoded)
        probabilities = torch.softmax(output, dim=1)
        _, predicted = torch.max(output, 1)
    
    return predicted.item(), probabilities[0].numpy()

# Тестируем на новых последовательностях
test_sequences = [
    "ATGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG",  # Начинается с ATG
    "CGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG",  # Не начинается с ATG
    "ATGAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA",  # ATG + много A
]

print("Предсказания BiLSTM модели:\n")
for seq in test_sequences:
    pred, probs = predict_sequence(model_bilstm, seq)
    print(f"Последовательность: {seq[:20]}...")
    print(f"  Предсказание: {'Has ATG' if pred == 1 else 'No ATG'}")
    print(f"  Вероятности: No ATG={probs[0]:.4f}, Has ATG={probs[1]:.4f}")
    print()

## 15. Визуализация внимания (attention) — бонус

Хотя базовые LSTM не имеют встроенного механизма attention, мы можем посмотреть на активации скрытых состояний.

In [ ]:
# Модель с доступом к скрытым состояниям
class LSTMWithHidden(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMWithHidden, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Получаем все скрытые состояния
        out, _ = self.lstm(x, (h0, c0))
        predictions = self.fc(out[:, -1, :])
        
        return predictions, out

# Визуализируем активации
model_viz = LSTMWithHidden(input_size, hidden_size, output_size)
test_seq = encode_sequence("ATGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG").unsqueeze(0)

model_viz.eval()
with torch.no_grad():
    _, hidden_states = model_viz(test_seq)

# Визуализируем нормы скрытых состояний
hidden_norms = torch.norm(hidden_states[0], dim=1).numpy()

plt.figure(figsize=(14, 4))
plt.plot(hidden_norms, linewidth=2)
plt.title('Hidden State Activations по позициям в последовательности')
plt.xlabel('Позиция в последовательности')
plt.ylabel('Норма скрытого состояния')
plt.grid(True)
plt.show()

# Heatmap активаций
plt.figure(figsize=(14, 6))
plt.imshow(hidden_states[0].numpy().T, aspect='auto', cmap='viridis')
plt.colorbar(label='Activation')
plt.title('Heatmap скрытых состояний LSTM')
plt.xlabel('Позиция в последовательности')
plt.ylabel('Нейрон скрытого слоя')
plt.show()

## 16. Дополнительные улучшения и техники

### Регуляризация:
- **Dropout** — случайное отключение нейронов для предотвращения переобучения
- **Weight decay** (L2 регуляризация)
- **Gradient clipping** — обрезка градиентов для стабильности обучения

### Другие архитектуры:
- **GRU** (Gated Recurrent Unit) — упрощенная версия LSTM
- **Attention механизмы** — фокусировка на важных частях последовательности
- **Transformer** — современная архитектура на основе self-attention (BERT, GPT)

In [ ]:
# Пример LSTM с dropout
class LSTMWithDropout(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.3):
        super(LSTMWithDropout, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                           batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

print("Модель с dropout создана")
model_dropout = LSTMWithDropout(input_size, hidden_size, output_size)
print(model_dropout)

## 17. Реальные применения в биологии

### Задачи, где RNN/LSTM особенно эффективны:

1. **Предсказание вторичной структуры белков** — последовательность аминокислот → вторичная структура (спираль, лист, петля)

2. **Аннотация генома** — идентификация функциональных элементов в последовательностях ДНК

3. **Предсказание сайтов связывания** — где транскрипционные факторы связываются с ДНК

4. **Анализ экспрессии генов** — временные ряды уровней экспрессии

5. **Классификация белковых последовательностей** — определение семейств и функций

6. **Предсказание модификаций РНК** — m6A, псевдоуридин и т.д.

7. **Дизайн лекарств** — генерация молекул с желаемыми свойствами (используя SMILES строки)

### Популярные библиотеки и инструменты:
- **BioPython** — работа с биологическими последовательностями
- **DeepChem** — машинное обучение для химии и биологии
- **ProtTrans** — трансформеры для анализа белков
- **ESM** (Evolutionary Scale Modeling) — предобученные модели для белков от Meta

## 18. Задания для самостоятельной работы

1. **Измените архитектуру**: попробуйте разные значения `hidden_size`, `num_layers`, добавьте dropout

2. **Усложните задачу**: вместо бинарной классификации создайте задачу с несколькими классами (например, классификация типов промоторов)

3. **Используйте реальные данные**: загрузите настоящие последовательности ДНК/белков из баз данных (GenBank, UniProt)

4. **Реализуйте GRU**: создайте модель на основе GRU и сравните с LSTM

5. **Добавьте attention**: реализуйте простой механизм attention для визуализации важных позиций

6. **Transfer learning**: используйте предобученную модель (например, из ProtTrans) и дообучите на своих данных

7. **Sequence-to-sequence**: реализуйте модель для предсказания структуры по последовательности (многоклассовая классификация для каждой позиции)

## 19. Полезные ресурсы

### Документация:
- [PyTorch RNN Tutorial](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)
- [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

### Статьи:
- Hochreiter & Schmidhuber (1997) "Long Short-Term Memory"
- Cho et al. (2014) "Learning Phrase Representations using RNN Encoder-Decoder"

### Биоинформатика:
- Alipanahi et al. (2015) "Predicting the sequence specificities of DNA- and RNA-binding proteins by deep learning"
- Rives et al. (2021) "Biological structure and function emerge from scaling unsupervised learning to 250 million protein sequences"

## Заключение

В этом notebook мы изучили:
- Основы RNN и их применение к биологическим последовательностям
- Архитектуру LSTM и решение проблемы исчезающего градиента
- Bidirectional LSTM для учета двунаправленного контекста
- Практическую реализацию на PyTorch
- Техники улучшения моделей (dropout, regularization)
- Реальные применения в биоинформатике

**Следующие шаги:**
- Попробуйте применить эти методы к вашим собственным данным
- Изучите современные архитектуры (Transformers, Attention)
- Используйте предобученные модели для transfer learning
- Экспериментируйте с разными типами последовательностей (ДНК, РНК, белки)